In [228]:
import pandas as pd
import numpy as np
import os
import time
# 读取数据
path = r"C:\Users\65680\Desktop\熵值法"
data = pd.ExcelFile(r"C:\Users\65680\Desktop\熵值法\第五章.xlsx")
df = data.parse("品种筛选")
df_c = data.parse("品种筛选")

In [229]:
df.set_index("品种",inplace=True)
df_c.set_index("品种",inplace=True)
# 均值采取的列
n = list(df.columns)
# 熵值法采取的列
df1 = df_c[["镉", "镉-1","小区", "大区", "成本"]]
k = list(df1.columns)

In [230]:
for i in n:
    mean = np.mean(df[i])
    df[i] = df[i]-mean
df[df>=0] = 0
df[df<0] = 1

In [231]:
for i in k:
    # 获取各个指标的最大值和最小值
    Max = np.max(df1[i])
    Min = np.min(df1[i])
    if i in ["小区", "大区"]:
        df1[i] = (df1[i] - Min) / (Max - Min)
    else:
        df1[i] = (Max - df1[i]) / (Max - Min)
df1.replace(0, 0.0001, inplace=True)


<ipython-input-231-3c4f7072149c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[i] = (Max - df1[i]) / (Max - Min)
D:\Program Files\Python\lib\site-packages\pandas\core\frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [232]:
for i in k:
    # 计算指标总和
    Sum = np.sum(df1[i])
    # 计算各地区某一指标占比
    df1[i] = df1[i] / Sum
# 地区总数
m = len(df1)
E = []
# 计算信息熵值
for i in k:
    K = 1 / np.log(m)
    e = - K * np.sum(df1[i] * np.log(df1[i]))
    E.append(e)
# 转换为数组形式
E = np.array(E)
# 计算效用价值
D = 1 - E
W = D / np.sum(D)
# 转换形式
W = np.array([W])
# 保存 权重 为excel格式
W1 = pd.DataFrame(W.T, index=k)
U = []
for i in range(1, len(df1) + 1):
    # 获取样本各个指标的值
    y = df1[i - 1:i].values
    u1 = y * W * 100
    # 转换为列表
    u1 = u1.tolist()
    u1 = u1[0]

    # 计算样本综合得分
    ## 因前文构建了数据权重矩阵，故最后综合得分总和为100
    u = np.sum(y * W) * 100
    # 转换为列表
    u = np.array([u])
    u = u.tolist()
    # 各指标得分 和 综合得分
    u = u1 + u
    U.append(u)
# 获取样本列表
area = list(df1.index)
# 生成数据框
U = pd.DataFrame(U, index=area)
# 重新设置列名称
U.columns = k + ['综合得分']
# 为各个指标得分排名
for i in k:
    i1 = i + '  排名'
    U[i1] = U[i].rank(ascending=False)
# 为样本综合得分排名
U['综合得分  排名'] = U['综合得分'].rank(ascending=False)

<ipython-input-232-50c975fe052c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[i] = df1[i] / Sum


In [233]:
since = df[df["镉"]==0]
since = pd.DataFrame(since)

In [234]:
since2 = df[df["镉-1"]==0]
since2 = pd.DataFrame(since2)


In [235]:
wr = pd.ExcelWriter(os.path.join(path,"品种筛选.xlsx"))
since.to_excel(wr,sheet_name="小区")
since2.to_excel(wr,sheet_name="大区")
W1.to_excel(wr,sheet_name="熵值法权重")
U.to_excel(wr,sheet_name="熵值法综合得分")
wr.save()
wr.close()